# Experiment 004: Strict Validation + Conservative Optimization

Critical fix: exp_002 FAILED due to overlaps in group 004 (area 3.47e-13).

Approach:
1. ZERO tolerance overlap checking (intersects AND NOT touches = overlap)
2. Start from grid placement (known to work on LB)
3. Apply optimization only if it passes strict validation
4. Fall back to grid placement if optimization creates overlaps

In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time
from numba import njit

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')
SCALE = float(scale_factor)

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
# Tree polygon template (numba-compatible)
@njit
def make_polygon_template():
    tw = 0.15; th = 0.2; bw = 0.7; mw = 0.4; ow = 0.25
    tip = 0.8; t1 = 0.5; t2 = 0.25; base = 0.0; tbot = -th
    x = np.array([0, ow/2, ow/4, mw/2, mw/4, bw/2, tw/2, tw/2, -tw/2, -tw/2, -bw/2, -mw/4, -mw/2, -ow/4, -ow/2], np.float64)
    y = np.array([tip, t1, t1, t2, t2, base, base, tbot, tbot, base, base, t2, t2, t1, t1], np.float64)
    return x, y

TX, TY = make_polygon_template()

@njit
def get_side_length_fast(xs, ys, degs, tx, ty):
    n = len(xs); V = len(tx)
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    return max(mxx - mnx, mxy - mny)

@njit
def score_group(xs, ys, degs, tx, ty):
    n = len(xs)
    side = get_side_length_fast(xs, ys, degs, tx, ty)
    return side * side / n

print("Fast numba functions defined")

Fast numba functions defined


In [3]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._build_polygon()
    
    def _build_polygon(self):
        trunk_w = Decimal('0.15'); trunk_h = Decimal('0.2')
        base_w = Decimal('0.7'); mid_w = Decimal('0.4'); top_w = Decimal('0.25')
        tip_y = Decimal('0.8'); tier_1_y = Decimal('0.5'); tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0'); trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))
    
    def update_position(self, new_x, new_y, new_angle=None):
        self.center_x = Decimal(str(new_x))
        self.center_y = Decimal(str(new_y))
        if new_angle is not None:
            self.angle = Decimal(str(new_angle))
        self._build_polygon()

print("ChristmasTree class defined")

ChristmasTree class defined


In [4]:
def has_overlap_zero_tolerance(trees):
    """ZERO TOLERANCE overlap check.
    
    If two polygons intersect AND don't just touch, it's an overlap.
    This is the strictest possible check.
    """
    if len(trees) <= 1:
        return False, []
    
    polygons = [t.polygon for t in trees]
    overlap_details = []
    
    for i in range(len(polygons)):
        for j in range(i + 1, len(polygons)):
            poly_i = polygons[i]
            poly_j = polygons[j]
            
            # ZERO TOLERANCE: if they intersect and don't just touch, it's overlap
            if poly_i.intersects(poly_j) and not poly_i.touches(poly_j):
                # Calculate area for reporting
                intersection = poly_i.intersection(poly_j)
                area = intersection.area / (SCALE ** 2)
                overlap_details.append((i, j, area))
    
    return len(overlap_details) > 0, overlap_details

def has_overlap_single_zero_tolerance(trees, tree_idx):
    """Check if a single tree overlaps with any other - ZERO TOLERANCE."""
    if len(trees) <= 1:
        return False
    
    target_poly = trees[tree_idx].polygon
    
    for i, tree in enumerate(trees):
        if i == tree_idx:
            continue
        if target_poly.intersects(tree.polygon) and not target_poly.touches(tree.polygon):
            return True
    return False

print("ZERO TOLERANCE overlap functions defined")

ZERO TOLERANCE overlap functions defined


In [5]:
def find_best_trees_grid(n):
    """Grid-based placement - KNOWN TO WORK on Kaggle LB."""
    best_score = float('inf')
    best_trees = None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
                
            all_trees = []
            rest = n
            r = 0
            
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = Decimal('0') if r % 2 == 0 else Decimal('0.35')
                
                if r % 2 == 0:
                    y = Decimal(str(r // 2)) * Decimal('1.0')
                else:
                    y = Decimal('0.8') + Decimal(str((r - 1) // 2)) * Decimal('1.0')
                
                row_trees = [
                    ChristmasTree(
                        center_x=Decimal('0.7') * Decimal(str(i)) + x_offset,
                        center_y=y,
                        angle=angle
                    )
                    for i in range(m)
                ]
                all_trees.extend(row_trees)
                r += 1
            
            if len(all_trees) != n:
                continue
            
            xs = np.array([float(t.center_x) for t in all_trees])
            ys = np.array([float(t.center_y) for t in all_trees])
            degs = np.array([float(t.angle) for t in all_trees])
            score = score_group(xs, ys, degs, TX, TY)
            
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

print("Grid placement function defined")

Grid placement function defined


In [6]:
def local_search_conservative(trees, max_iters=100):
    """Conservative local search with ZERO TOLERANCE validation.
    
    Only accepts moves that pass strict validation.
    """
    if len(trees) <= 1:
        return trees, False
    
    xs = np.array([float(t.center_x) for t in trees])
    ys = np.array([float(t.center_y) for t in trees])
    degs = np.array([float(t.angle) for t in trees])
    
    best_side = get_side_length_fast(xs, ys, degs, TX, TY)
    improved_any = False
    steps = [0.02, 0.01, 0.005]
    
    for iteration in range(max_iters):
        improved = False
        cx, cy = xs.mean(), ys.mean()
        
        for i in range(len(trees)):
            dx = cx - xs[i]
            dy = cy - ys[i]
            dist = math.sqrt(dx * dx + dy * dy)
            
            if dist < 1e-6:
                continue
            
            dx /= dist
            dy /= dist
            
            for step in steps:
                orig_x, orig_y = xs[i], ys[i]
                
                xs[i] = orig_x + dx * step
                ys[i] = orig_y + dy * step
                trees[i].update_position(xs[i], ys[i])
                
                # ZERO TOLERANCE check
                if not has_overlap_single_zero_tolerance(trees, i):
                    new_side = get_side_length_fast(xs, ys, degs, TX, TY)
                    if new_side < best_side - 1e-12:
                        best_side = new_side
                        improved = True
                        improved_any = True
                        break
                    else:
                        xs[i] = orig_x
                        ys[i] = orig_y
                        trees[i].update_position(xs[i], ys[i])
                else:
                    xs[i] = orig_x
                    ys[i] = orig_y
                    trees[i].update_position(xs[i], ys[i])
        
        if not improved:
            break
    
    return trees, improved_any

print("Conservative local search function defined")

Conservative local search function defined


In [7]:
def fractional_translation_conservative(trees, max_iters=50):
    """Conservative fractional translation with ZERO TOLERANCE."""
    if len(trees) <= 1:
        return trees, False
    
    xs = np.array([float(t.center_x) for t in trees])
    ys = np.array([float(t.center_y) for t in trees])
    degs = np.array([float(t.angle) for t in trees])
    
    best_side = get_side_length_fast(xs, ys, degs, TX, TY)
    improved_any = False
    
    # Smaller step sizes for safety
    frac_steps = [0.001, 0.0005, 0.0002]
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
    
    for iteration in range(max_iters):
        improved = False
        
        for i in range(len(trees)):
            for step in frac_steps:
                for dx, dy in directions:
                    orig_x, orig_y = xs[i], ys[i]
                    
                    xs[i] = orig_x + dx * step
                    ys[i] = orig_y + dy * step
                    trees[i].update_position(xs[i], ys[i])
                    
                    if not has_overlap_single_zero_tolerance(trees, i):
                        new_side = get_side_length_fast(xs, ys, degs, TX, TY)
                        if new_side < best_side - 1e-12:
                            best_side = new_side
                            improved = True
                            improved_any = True
                        else:
                            xs[i] = orig_x
                            ys[i] = orig_y
                            trees[i].update_position(xs[i], ys[i])
                    else:
                        xs[i] = orig_x
                        ys[i] = orig_y
                        trees[i].update_position(xs[i], ys[i])
        
        if not improved:
            break
    
    return trees, improved_any

print("Conservative fractional translation function defined")

Conservative fractional translation function defined


In [8]:
# Generate solutions with STRICT validation
start_time = time.time()

solutions = {}
scores = {}
fallback_count = 0

for n in range(1, 201):
    # Special case: N=1 at 45 degrees
    if n == 1:
        trees = [ChristmasTree(center_x='0', center_y='0', angle='45')]
    else:
        # Start with grid placement (KNOWN TO WORK)
        _, grid_trees = find_best_trees_grid(n)
        
        # Make a copy for optimization
        trees = [ChristmasTree(str(t.center_x), str(t.center_y), str(t.angle)) for t in grid_trees]
        
        # Try local search
        trees, improved1 = local_search_conservative(trees, max_iters=30)
        
        # Try fractional translation
        trees, improved2 = fractional_translation_conservative(trees, max_iters=20)
        
        # CRITICAL: Final validation with ZERO TOLERANCE
        has_overlap, details = has_overlap_zero_tolerance(trees)
        
        if has_overlap:
            # FALLBACK to grid placement
            trees = grid_trees
            fallback_count += 1
            if n <= 20:  # Only print for small N
                print(f"  N={n}: Optimization created overlaps, falling back to grid")
    
    solutions[n] = trees
    
    # Calculate score
    xs = np.array([float(t.center_x) for t in trees])
    ys = np.array([float(t.center_y) for t in trees])
    degs = np.array([float(t.angle) for t in trees])
    scores[n] = score_group(xs, ys, degs, TX, TY)
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f"N={n:3d}: score={scores[n]:.6f}, elapsed={elapsed:.1f}s")

total_time = time.time() - start_time
print(f"\nTotal time: {total_time:.1f}s")
print(f"Fallback to grid: {fallback_count} configurations")

N= 20: score=0.496125, elapsed=2.9s


N= 40: score=0.441000, elapsed=10.5s


N= 60: score=0.450667, elapsed=29.6s


N= 80: score=0.442531, elapsed=58.5s


N=100: score=0.396900, elapsed=103.6s


N=120: score=0.408333, elapsed=168.0s


N=140: score=0.385875, elapsed=255.5s


N=160: score=0.405016, elapsed=369.1s


N=180: score=0.392000, elapsed=512.3s


N=200: score=0.405000, elapsed=689.4s

Total time: 689.4s
Fallback to grid: 0 configurations


In [ ]:
# Calculate total score
total_score = sum(scores[n] for n in range(1, 201))
print(f"Total score: {total_score:.6f}")
print(f"Target score: 68.947559")
print(f"Gap: {total_score - 68.947559:.6f}")
print(f"\nComparison:")
print(f"  Grid placement (exp_001): 88.33")
print(f"  This experiment: {total_score:.2f}")

In [ ]:
# FINAL VALIDATION - Check ALL groups with ZERO TOLERANCE
print("\n=== FINAL VALIDATION (ZERO TOLERANCE) ===")
overlap_issues = []

for n in range(1, 201):
    trees = solutions[n]
    has_overlap, details = has_overlap_zero_tolerance(trees)
    if has_overlap:
        overlap_issues.append((n, details))
        print(f"  N={n}: OVERLAP DETECTED! {details}")

if overlap_issues:
    print(f"\n!!! CRITICAL: Overlaps found in {len(overlap_issues)} groups !!!")
    print("DO NOT SUBMIT - need to fix overlaps")
else:
    print("\n=== ALL CONFIGURATIONS VALID - SAFE TO SUBMIT ===")

In [ ]:
# Create submission
def to_str(x):
    return f"s{round(float(x), 6)}"

rows = []
for n in range(1, 201):
    trees = solutions[n]
    assert len(trees) == n, f"Expected {n} trees, got {len(trees)}"
    for i_t, tree in enumerate(trees):
        rows.append({
            "id": f"{n:03d}_{i_t}",
            "x": to_str(tree.center_x),
            "y": to_str(tree.center_y),
            "deg": to_str(tree.angle),
        })

submission = pd.DataFrame(rows)
print(f"Submission shape: {submission.shape}")
print(submission.head(10))

In [ ]:
# Save submission
submission.to_csv('/home/submission/submission.csv', index=False)
submission.to_csv('/home/code/experiments/004_strict_validation/submission.csv', index=False)
print("Submission saved!")
print(f"\nFinal score: {total_score:.6f}")